In [31]:
### Import packages ###
import numpy as np
import pandas as pd
import itertools

# Input Data Set 

In [32]:
# Data =["Auto"]
# Data =["BostonHousing"]
# Data =["CPS"]
Data =["Simulate"]
# Data =["Yacht"]


# Input Other Parameters

In [33]:
# Directory
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Define parameters
Seed = range(0,3)
TestProportion = [0.2]
CandidateProportion = [0.8]
SelectorType = ["PassiveLearning", "GSxFunction", "GSyFunction", "iGSFunction"]
ModelType = ["LinearRegressionFunction", "RidgeRegressionFunction", "RandomForestRegressorFunction"]
DataArgs = [{"N": 1000, "K": 5}]
SelectorArgs = [{"df_Candidate" : "df_Candidate"}]
ModelArgs = [{"alpha_val": 0.1,
              "n_estimators" :100,
              "Seed" : 1}]

### Combinations ###
Combinations = list(itertools.product(
    Seed, Data, TestProportion, CandidateProportion, SelectorType, ModelType, DataArgs, SelectorArgs, ModelArgs
))

### Data Frame ###
df = pd.DataFrame(Combinations,
                  columns = ["Seed", 
                  "Data", 
                  "TestProportion", 
                  "CandidateProportion", 
                  "SelectorType", 
                  "ModelType", 
                  "DataArgs", 
                  "SelectorArgs", 
                  "ModelArgs"])



In [34]:
### Job Name ###
ReplacementCharacters = {":": "",  " ": "", "{": "", "}": "", """: "", ",": "_", """: "", "[": "", "]": ""}
DataArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(DataArgs)])
ModelArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
SelectorArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
ModelTypeReplacement = str(ModelType).replace("Function","")
DataReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(Data)])

df["JobName"] = ("Seed" + df["Seed"].astype(str) +
                 "_Data" + df["Data"].astype(str) +
                 "_TP" + df["TestProportion"].astype(str) +
                 "_CP" + df["CandidateProportion"].astype(str) +
                 "_ST" + df["SelectorType"].astype(str) +
                 "_MT" + df["ModelType"].astype(str)
                #  DataArgsReplacement + "_" +
                #  ModelArgsReplacement
                 )

### Output Name ###
df["JobName"] = df["JobName"].str.replace("Function", "", regex=False)
df["Output"] =  df["Data"].astype(str) + "/" + df["ModelType"].astype(str) + "/Raw/" + df["JobName"]+".pkl"
df["Output"] = df["Output"].str.replace("Function", "", regex=False)


In [35]:
# Define the columns to process
columns_to_process = ['DataArgs', 'SelectorArgs', 'ModelArgs']
for col in columns_to_process:
    df[col] = "'" + df[col].apply(lambda x: str(x).replace("'", '"') if pd.notna(x) else x) + "'"


In [36]:
### Save Parameter Vector ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data[0]) + ".csv"
df.to_csv(output_path, index=False)